In [2]:
from surveys import Filter, BoxcarFilter, Survey, CASTOR_Default
import astropy.units as u
from astropy.table import Table
from numpy.random import default_rng
from numpy import log, log10
from scipy.integrate import simpson

# Notebook to generate Mock CASTOR photmetry

This notebook walks through the steps of generating mock CASTOR photometry given specific instrument and survey parameters. It uses galaxies from the COSMOS 2020 Farmer catalog. Mock magnitudes have already been pre-generated for other telescopes already using the LePhare best fit templates and applying color-corrections from nearby COSMOS filters. We assume that these represent the true magnitudes of the galaxies, and then perturb the photometry (in flux space) using the noise as the standard deviation of a normal distribution. From the pre-made catalog, this notebook can quickly make mock photometry for CASTOR with various survey/instrument parameters.

## Step 1: Define Castor Filters

The first step is to define what filters CASTOR will use. The pre-made mock catalogs have photometry for the default uv, u, and g filters (downloaded from Forecastor github accessed on May 20, 2022). You can load any new or modified filters and photometry in those new filters will be determined from the default filters using a color correction from the different zero point magnitudes. Note that this assumes the new filters will be relatively close in central wavelength and breadth.

Filter transmission curves can either be loaded from files, or parameterized. The example below uses new filters in uv and u that where divided in half with an exterior long-pass filter. The g filter is a simple square filter.

In [3]:
# define Castor Filter Transmission Curves

test_filt = [Filter(filename="FTCs/passband_castor.uv_split_lp", name="castor_uv_test", wlunit=u.um),
             Filter(filename="FTCs/passband_castor.u_split_lp", name="castor_u_test", wlunit=u.um),
             BoxcarFilter(center=5000, bandwidth=1620, transmission=0.6, name="castor_g_test", wlunit=u.AA)
            ]


## Step 2: Define Survey Strategy and Telescope Parameters

Now, we have to define the instrument details and observing strategy CASTOR will use. In the example below, the mirror diameter, dark current, read noise, etc. are all from Forecastor's castor_etc.parameters file. The sky background brightness was also made using forecastor and adding average geocoronal emission to the uv band. Note that the geocoronal emission line has a wavelength of 2471 Angstroms. If the filter you are using does not cover this, you should adjust the sky brightness in uv.

You need to define Nexp and texp, the number of exposures and the time per exposure in seconds. These can either be one value that apply to all filters, or an array with different values for each filter.

Two different methods of calculating photometric noise are provided: 'm5' and 'ext'

The 'm5' method uses Equation 5 of Ilbert et al. 2019:

$$ 
\sigma^2 = (0.04 - \gamma)x + \gamma{x^2}
$$

where $x = 10^{(m - m_5)}$, $m_5$ is the five sigma depth in a filter, and $\gamma$ depends on the sky background and other noise sources.

This method is an approximation for calculating photometric uncertainties in point sources given a survey depth. It is the easiest to use, but will underestimate the uncertainty in extended sources. When using this mode, you can either provide 5-sigma depths in each filter, or they can be omitted and will be internally calculated. User provided depths will override the automatic calculations.

The second mode is 'ext'. This will perform a more detailed photometric uncertainty calculation, taking into account the size of the objects being observed. The galaxy's FWHM is convolved with the telescope's PSF, and an aperture diameter of 1.346 times the convolved FWHM is used (see LSST LSE-40 Eq. 19). From there, the number of pixels ($N_{pix}$) in the aperture is determined. The signal-to-noise ratio is then briefly:

$$
SNR = \frac{e_{source}}{  \sqrt{e_{source} + e_{sky} + e_{dark} + N_{pix}N_{exp}\sigma^2_{read}} }
$$

where $e_{source}$ and $e_{sky}$ are the total electron counts in the aperture from the galaxy source and the sky respectively, found using the total exposure time and the magnitude zeropoints, $e_{dark}$ is the total number of electrons generated from the dark current multiplied by $N_{pix}$ and the total exposure time, and $\sigma_{read}$ is the read noise in electrons per exposure.

The 'ext' and 'm5' methods have been tested to show they give the same uncertainties for point sources. The 'ext' mode will be more accurate, but only if the instrument parameters such as dark current and read noise and mirror diameter, etc., are also accurate.

In [4]:
# Choose extended or m5 photometric catalog
# MODE = 'm5'
MODE = 'ext' # recommended mode

test_survey = Survey( # you can leave out the five sigma depth and it will be automatically calculated
           mirror_diam=100, dark_current=1e-4, read_noise=2.0, pixel_scale=0.1, 
           psf_fwhm=[0.15] * len(test_filt), filters=test_filt,
           Nexp=[8, 8, 6], texp=250, sky_mag=[25.24, 24.24, 22.59],
           min_aper_radius=2
          )

print(test_survey.five_sigma_depth)


{'castor_uv_test': 26.657455960658222, 'castor_u_test': 27.344479940851798, 'castor_g_test': 26.884888404194324}


## Step 3: Read in default catalog and choose limits

In this step, the pre-made default catalog is read in (either the ext or m5 version depending on what you chose above.

Since the goal is to calculate photometric redshifts, you may not want to use the faintest objects and instead limit yourself to less-noisy objects. You can optionally add a mask here to reduce the size of the default catalog (~700 000 objects)

In [4]:
default_cat_file = "%s_phot.fits" % MODE
cat = Table.read(default_cat_file)
default_survey = CASTOR_Default()
default_filt = default_survey.filters

# remove default mag and magerr columns to avoid potential clashes
cat.remove_columns([f.name + '_MAG' for f in default_filt] +
                   [f.name + '_MAGERR' for f in default_filt])

# Define mask
MASK = None
# Example mask to limit to LSST i < 27.1
# MASK = def_cat['LSST_i_MAG'] < 27.1

if MASK is not None:
    cat = def_cat[MASK]                                                                   

## Step 4: Choose the name for the output catalog

In [5]:
OUTFILE = "test_mock_phot.fits"

## Step 5: Make the mock photometry catalog

Finally, run the cell below to create your mock photometry catalog. This can then be put into your photometric redshift code of choice.

In [6]:
rng = default_rng()

for f in test_filt:
    # find closest default CASTOR filter
    close = default_filt[0]
    for g in default_filt:
        if abs(g.center - f.center) < abs(close.center - f.center):
            close = g
    
    # find new unperturbed filter mags from unperturbed castor mags
    # uses definitions m_AB = -2.5 * log(n_e) + ZP
    # and n_e = A * f_v / h * int(response / lambda, lambda)
    # this assumes f_v is constant
    zp_offset = test_survey.zpts[f.name] - default_survey.zpts[close.name]
    numerator = simpson(f.response / f.wl, f.wl)
    denominator = simpson(close.response / close.wl, close.wl)
    area_term = -2.5 * log(numerator / denominator)
    print(f.name, "offset: ", area_term + zp_offset)
    new_mag = cat[close.name + '_UNPMAG'] + area_term  + zp_offset
    
    # calculate magnitude uncertainty
    if MODE == 'ext':
        new_magerr = test_survey.calc_mag_err(new_mag, f.name, fwhm=2 * cat['hlr'])
    elif MODE == 'm5':
        new_magerr = test_survey.calc_mag_err(new_mag, f.name, mode='m5')
    
    # perturb flux within noise and convert back to mag 
    flux = 10**(-0.4 * (new_mag + 48.6))
    snr = 2.5 / log(10) / new_magerr
    noise = flux / snr
    error = rng.standard_normal(noise.size) * noise
    obs_flux = flux + error
    obs_mag = -2.5 * log10(obs_flux) - 48.6
        
    cat.add_column(obs_mag, name=f.name + '_MAG')
    cat.add_column(new_magerr, name=f.name + '_MAGERR')
    

cat.remove_columns([f.name + '_UNPMAG' for f in default_filt])

cat.write(OUTFILE, overwrite=True)

castor_uv_test offset:  1.1346273842078123
castor_u_test offset:  0.057732985765189926


/tmp/ipykernel_304/456521459.py:33: RuntimeWarning: invalid value encountered in log10
  obs_mag = -2.5 * log10(obs_flux) - 48.6


castor_g_test offset:  -0.10545144308151547
